In [9]:


import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [61]:
def parsing_dscTable(table_element):
    html_table = table_element.get_attribute('outerHTML')
    table = pd.read_html(html_table)[0]
    table = table[table['E value'] < 0.001]
    
    msg = []
    if len(table) == 0:
        msg.append('-')
    else:
        table_cut = table[:20][
            (~table.Description.str.contains('hypothetical protein', regex=False))
        ]

        if len(table_cut) == 0:
            msg.append('hp')
        else:
            for idx, row in table.iterrows():
                if (
                    (('hypothetical protein' not in row['Description'])
                     and ("conserved hypothetical protein" not in row['Description'])
                     and ("unnamed protein product" not in row['Description'])
                    ) 
                    and len(msg) < 3):

                    msg.append(f"{row['Description']};{row['E value']}")
                    
    return msg

def parsing_dscTable(table_element):
    html_table = table_element.get_attribute('outerHTML')
    table_results = pd.read_html(html_table)[0]
    # Удаление всего лишнего из таблицы
    table_results = table_results.iloc[1:, 1:].dropna()[::2]
    # Оставляем только данные где E-value < 0.001
    table_results['E_value_float'] = table_results[5].astype(float)
    table_results = table_results[table_results['E_value_float'] < 0.001]
    
    msg = []
    if len(table_results) == 0:
        msg.append('-')
    else:
        for idx, row in table_results.iterrows():
            if len(msg) < 3:
                msg.append(f'{row[1]};{row[5]}')
    
    return msg

In [10]:
table = pd.read_excel('CP012104_1_-_RAST.xls', sheet_name='Sheet1')

In [81]:
driver = webdriver.Chrome()
driver.wait = WebDriverWait(driver, 5)
driver.get(f"https://blast.ncbi.nlm.nih.gov/"
           f"Blast.cgi?PROGRAM=blastp&PAGE_TYPE=BlastSearch&LINK_LOC=blasthome")

box = driver.wait.until(EC.presence_of_element_located(
            (By.NAME, "QUERY")))
button = driver.wait.until(EC.element_to_be_clickable(
            (By.CLASS_NAME, "blastbutton")))
organism = driver.wait.until(EC.presence_of_element_located(
            (By.NAME, "EQ_MENU")))
add_options = driver.wait.until(EC.element_to_be_clickable(
            (By.XPATH, '//*[@id="algPar"]/span[2]')))
psi_blast = driver.wait.until(EC.element_to_be_clickable(
            (By.XPATH, '//*[@id="psiBlast"]')))

box.send_keys(table.aa_sequence[61])
organism.send_keys(f'Viruses (taxid:10239)')
psi_blast.click()
#add_options.click()
button.click()
input() # !!!Тут надо подумать как грамотно сделать паузу!!!
#table_result = driver.wait.until(EC.presence_of_element_located(
            #(By.XPATH, '//*[@id="dscTable"]')))






456


'456'

In [83]:
graphic_summary = driver.wait.until(EC.element_to_be_clickable(
            (By.XPATH, '//*[@id="btnGrph"]')))
graphic_summary.click()

In [84]:
cddInfo = driver.wait.until(EC.element_to_be_clickable(
            (By.XPATH, '//*[@id="cddInfo"]/a/img')))
cddInfo.click()

In [110]:
table_cddInfo = driver.wait.until(EC.presence_of_element_located(
            (By.XPATH, '//*[@id="std"]')))


In [109]:
table_results

,1,2,3,4,5,E_value_float
1,Sipho_tail,pfam05709,Phage tail protein; This family consists of se...,17-268,3.29e-24,3.290000e-24
7,YomH,COG4722,"Phage-related protein [Mobilome: prophages, tr...",1-264,1.39e-09,1.390000e-09
13,phi3626_gp14_N,TIGR01633,"putative phage tail component, N-terminal doma...",35-132,2.34e-06,2.340000e-06


In [111]:
msg

['Sipho_tail;3.29e-24', 'YomH;1.39e-09', 'phi3626_gp14_N;2.34e-06']

In [86]:
# открытые табы
driver.window_handles

['CDwindow-39C2A9E1A726CF15B31D88B8B03B9676',
 'CDwindow-3832C108FCA41818F265454200ED3A5D']

In [87]:
# перемещание по табам
driver.switch_to_window(driver.window_handles[1])

/home/andr/.pyenv/versions/3.7.4/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: use driver.switch_to.window instead
  


In [80]:
html_table

'<table id="std" class="disptbl">\n\t\t\t\t\t\t\t\t\t\t<tbody><tr class="ttl">\n\t\t\t\t\t\t\t\t\t\t\t<td class="mswitch" onclick="hMasterSwitchDetail(this);"><div></div></td>\n\t\t\t\t\t\t\t\t\t\t\t<td>Name</td>\n\t\t\t\t\t\t\t\t\t\t\t<td>Accession</td>\n\t\t\t\t\t\t\t\t\t\t\t<td class="descr">Description</td>\n\t\t\t\t\t\t\t\t\t\t\t<td>Interval</td>\n\t\t\t\t\t\t\t\t\t\t\t<td>E-value</td>\n\t\t\t\t\t\t\t\t\t\t</tr>\n\t\t\t\t\t\t\t\t\t\t<tr class="entry" id="std_0_0"><td class="switch" onclick="hSwitchDetail(this);"><div></div></td><td>DUF4236</td>\n<td><a href="//www.ncbi.nlm.nih.gov/Structure/cdd/cddsrv.cgi?ascbin=8&amp;maxaln=10&amp;seltype=2&amp;uid=pfam14020">pfam14020</a></td>\n<td class="descr"><div>Protein of unknown function (DUF4236); This family of proteins is functionally uncharacterized. ...</div></td>\n<td class="itvl">3-55</td>\n<td class="eval">4.55e-19</td>\n</tr>\n<tr class="detail backstage"><td class="switch"></td>\n<td colspan="5"><p>Protein of unknown function (D